In [1]:
import keras
keras.__version__

/home/baxter/tensorflow/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'2.1.6'

In [2]:
#Importing the external functions to read the image datasets
import build_cgd_train
import build_cgd_test
train_data, train_targets = build_cgd_train.read_train_data()
test_data, test_targets = build_cgd_test.read_test_data()

In [3]:
#Converting the data lists imported before into numpy arrays
import numpy as np

train_data = np.array(train_data)
train_targets = np.array(train_targets)
test_data = np.array(test_data)
test_targets = np.array(test_targets)

In [4]:
#Check the input shape for both train and test set
print('Train dataset shape:', train_data.shape)
print('Train targets shape:', train_targets.shape)
print('Test set shape:', test_data.shape)
print('Test targets shape:', test_targets.shape)

('Train dataset shape:', (4936, 150528))
('Train targets shape:', (4936, 5))
('Test set shape:', (177, 150528))
('Test targets shape:', (177, 5))


In [19]:
#Define the model architecture.
from keras import models
from keras import layers
from keras import optimizers
from keras.engine import topology

def build_model():
    # Because we will need to instantiate
    # the same model multiple times,
    # we use a function to construct it.
    model = models.Sequential()
    
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    
    model.add(layers.Conv1D(filters=64, kernel_size=5, strides=2, padding='same',
                            use_bias=True, kernel_initializer='random_uniform', 
                            bias_initializer='zeros'))
    model.add(layers.MaxPooling1D(pool_size=2, strides=2, padding='same'))
    
    model.add(layers.Conv1D(filters=128, kernel_size=3, strides=2, padding='same',
                            use_bias=True, kernel_initializer='random_uniform', 
                            bias_initializer='zeros'))
    model.add(layers.MaxPooling1D(pool_size=2, strides=2, padding='same'))
    
    model.add(layers.Conv1D(filters=128, kernel_size=3, strides=2, padding='same',
                            use_bias=True, kernel_initializer='random_uniform', 
                            bias_initializer='zeros'))
    
    model.add(layers.Conv1D(filters=128, kernel_size=3, strides=2, padding='same',
                            use_bias=True, kernel_initializer='random_uniform', 
                            bias_initializer='zeros'))
    
    model.add(layers.Conv1D(filters=256, kernel_size=3, strides=2, padding='same',
                            use_bias=True, kernel_initializer='random_uniform', 
                            bias_initializer='zeros'))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(5))
    
    adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, 
                       epsilon=None, decay=0.0, amsgrad=False)
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

In [20]:
#Train the model
model = build_model()
history = model.fit(train_data, train_targets, epochs=30, batch_size=1, verbose=1)

ValueError: Input 0 is incompatible with layer conv1d_7: expected ndim=3, found ndim=2

In [ ]:
model = build_model_v2()
    # Train the model (in silent mode, verbose=0)
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=1)

In [16]:
#Define custom loss function, metrics, and grasp_to_bboxes
import tensorflow as tf
from shapely.geometry import Polygon
import math
from keras import backend as K

def grasps_to_bboxes(grasp):
    #Order of input: x, y, tan, h, w
    bbox = []
    x = grasp[0]
    y = grasp[1]
    tan = grasp[2]
    h = grasp[3]
    w = grasp[4]
    
    theta = tf.atan(tan);
    
    bbox.append(x+((-w/2)*tf.cos(theta))-((-h/2)*tf.sin(theta)))
    bbox.append(y+((-h/2)*tf.cos(theta))+((-w/2)*tf.sin(theta)))
    bbox.append(x+((-w/2)*tf.cos(theta))-((h/2)*tf.sin(theta)))
    bbox.append(y+((h/2)*tf.cos(theta))+((-w/2)*tf.sin(theta)))
    bbox.append(x+((w/2)*tf.cos(theta))-((h/2)*tf.sin(theta)))
    bbox.append(y+((h/2)*tf.cos(theta))+((w/2)*tf.sin(theta)))
    bbox.append(x+((w/2)*tf.cos(theta))-((-h/2)*tf.sin(theta)))
    bbox.append(y+((-h/2)*tf.cos(theta))+((w/2)*tf.sin(theta)))
    
    bbox = np.array(bbox)
    return bbox
    
    
def rsse_loss_function(y_true, y_pred):
    #Order of output: x, y, tan, h, w
    tan_hat = y_pred[2]
    tan_hat_confined = tf.minimum(11., tf.maximum(-11., tan_hat))
    tan = y_true[2]
    tan_confined = tf.minimum(11., tf.maximum(-11., tan))
    gamma = tf.constant(10.)
    loss = tf.pow(y_pred[0] - y_true[0], 2) + tf.pow(y_pred[1] - y_true[1], 2) + gamma*tf.pow(tan_hat_confined - tan_confined, 2) + tf.pow(y_pred[3] - y_true[3], 2) + tf.pow(y_pred[4] - y_true[4], 2)

    return loss


def _floats_feature(v):
    if not isinstance(v, list):
        v = [v]
    return tf.train.Feature(float_list=tf.train.FloatList(value=v))

def rectangle_metric(y_true, y_pred):
    rec_true = grasps_to_bboxes(y_true)
    rec_pred = grasps_to_bboxes(y_pred)
    
    """poly_true = Polygon([(float(rec_true[0]), float(rec_true[1])), 
                         (float(rec_true[2]), float(rec_true[3])),
                         (float(rec_true[4]), float(rec_true[5])),
                         (float(rec_true[6]), float(rec_true[7]))
                        ])
    poly_pred = Polygon([(float(rec_pred[0]), float(rec_pred[1])),
                         (float(rec_pred[2]), float(rec_pred[3])),
                         (float(rec_pred[4]), float(rec_pred[5])),
                         (float(rec_pred[6]), float(rec_pred[7]))
                        ])"""
    
    poly_true = Polygon([(rec_true[0:2]), 
                        (rec_true[2:4]),
                        (rec_true[4:6]),
                        (rec_true[6:8])])
    poly_pred = Polygon([(rec_pred[0:2]), 
                        (rec_pred[2:4]),
                        (rec_pred[4:6]),
                        (rec_pred[6:8])])
    
    iou = poly_true.intersection(poly_pred).area / (poly_true.area +poly_pred.area - poly_true.intersection(poly_pred).area) 
    angle_diff = K.abs(K.atan(y_pred[2])*180/K.pi -K.atan(y_true[2])*180/K.pi)
    
    return iou, angle_diff

In [18]:
#iou, angle_diff = rectangle_metric(train_targets[0], train_targets[10])
#loss = rsse_loss_function(train_targets[0], train_targets[10])

In [19]:
#Define custom loss and metric functions
import tensorflow as tf
from shapely.geometry import Polygon

def rsse_loss_function(y_true, y_pred):
    #Order of output: x, y, tan, h, w
    tan_hat = y_pred[2]
    tan_hat_confined = tf.minimum(11., tf.maximum(-11., tan_hat))
    tan = y_true[2]
    tan_confined = tf.minimum(11., tf.maximum(-11., tan))
    gamma = 10
    loss = tf.reduce_sum(K.pow(y_pred[0] - y_true[0], 2) + tf.pow(y_pred[1] - y_true[1], 2) + gamma*tf.pow(tan_hat_confined - tan_confined, 2) + tf.pow(y_pred[3] - y_true[3], 2) + tf.pow(y_pred[4] - y_true[4], 2))
    
    return loss

def rectangle_metric(y_true, y_pred):
    poly_true = Polygon(float(grasp_to_rectangle(y_true)))
    poly_pred = Polygon(float(grasp_to_rectangle(y_pred)))
    
    iou = pply_true.intersection(poly_pred).area / (poly_true.area +poly_pred.area - poly_true.intersection(poly_pred).area) 
    angle_diff = np.abs(np.arctan(y_pred[2])*180/np.pi -np.arctan(y_true[2])*180/np.pi)
    
    return iou, angle_diff

In [48]:
#Re-define the model architecture after revised loss and metric functions.
from keras import models
from keras import layers

def build_model_v2():
    # Because we will need to instantiate
    # the same model multiple times,
    # we use a function to construct it.
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(5))
    model.compile(optimizer='rmsprop', loss=rsse_loss_function, metrics=[rectangle_metric])
    return model

In [49]:
import numpy as np

k = 5
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples,]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples,]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model_v2()
    # Train the model (in silent mode, verbose=0)
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=1)
    # Evaluate the model on the validation data
    val_loss, val_iou, val_angle_diff = model.evaluate(val_data, val_targets, verbose=1)
    all_scores.append(val_loss)

('processing fold #', 0)


TypeError: float() argument must be a string or a number